# Introduction

Here I will explain about the different creativity tests I implemented, as well as how to use the repo to test your own models. I will include code and hopefully plot some cool results.

# Text Modality

## Divergent Association Task (DAT)
This is a creativity test where a person/LLM is asked to name 10 words that are as different from each other as possible. \
Here is the actual prompt I ask the LLMs to answer:

''' \
**Please enter 10 words that are as different from each other as possible, \
in all meanings and uses of the words. Rules: Only single words in English. \
Only nouns (e.g., things, objects, concepts). No proper nouns (e.g., no specific people or places). \
No specialized vocabulary (e.g., no technical terms). \
Think of the words on your own (e.g., do not just look at objects in your surroundings). \
Make a list of these 10 words, a single word in each entry of the list. \
Do not write anything else, but the 10 words.** \
'''

Let's run a simple DAT with several gemini models to explore the effect of temperature on the models.

In [1]:
import os
import json
import numpy as np
from google import genai
from google.genai import types
from langchain_text_splitters import RecursiveCharacterTextSplitter
from embeddings import GloVe, calculate_dat_score
from request import Request, run_request
from scipy.stats import norm
from datetime import datetime

In [2]:
from creative_tests.DAT import DivergentAssociationTest

test = DivergentAssociationTest(
    models=["gemini-2.0-flash", "gemini-1.5-flash"],
    configs=[{"temperature": 0.5} ,{"temperature": 1}, {"temperature": 1.5}, {"temperature": 2}],
    repeats=10,
    delay=5,
)

In [3]:
test.run()

Number of API calls: 80
Estimated Time: 7.333333333333333
Requesting gemini-2.0-flash with config={temperature=0.5} (1)...


KeyboardInterrupt: 

In [1]:
import os
import json
import numpy as np
from google import genai
from google.genai import types
from langchain_text_splitters import RecursiveCharacterTextSplitter
from embeddings import GloVe, calculate_dat_score
from request import Request, run_request
from scipy.stats import norm
from datetime import datetime
from creative_tests.DAT import DivergentAssociationTest

test = DivergentAssociationTest(
    models=[
        "gemma-3-27b-it",
        "gemini-2.5-flash-preview-05-20",
        "gemini-2.5-flash-preview-04-17",
        "gemini-2.0-flash", 
        "gemini-1.5-flash",
        ],
    configs=[{"temperature": 0.5} ,{"temperature": 1}, {"temperature": 1.5}, {"temperature": 2}],
    repeats=20,
    delay=6,
)

test.run()

Number of API calls: 400
Estimated Time: 43.333333333333336
Requesting gemma-3-27b-it with config={temperature=0.5} (1)...
Requesting gemma-3-27b-it with config={temperature=0.5} (2)...
Requesting gemma-3-27b-it with config={temperature=0.5} (3)...
Requesting gemma-3-27b-it with config={temperature=0.5} (4)...
Requesting gemma-3-27b-it with config={temperature=0.5} (5)...
Requesting gemma-3-27b-it with config={temperature=0.5} (6)...
Requesting gemma-3-27b-it with config={temperature=0.5} (7)...
Requesting gemma-3-27b-it with config={temperature=0.5} (8)...
Requesting gemma-3-27b-it with config={temperature=0.5} (9)...
Requesting gemma-3-27b-it with config={temperature=0.5} (10)...
Requesting gemma-3-27b-it with config={temperature=0.5} (11)...
Requesting gemma-3-27b-it with config={temperature=0.5} (12)...
Requesting gemma-3-27b-it with config={temperature=0.5} (13)...
Requesting gemma-3-27b-it with config={temperature=0.5} (14)...
Requesting gemma-3-27b-it with config={temperature=0.

TypeError: float() argument must be a string or a real number, not 'NoneType'

In [1]:
import os
import json
import numpy as np
from google import genai
from google.genai import types
from langchain_text_splitters import RecursiveCharacterTextSplitter
from embeddings import GloVe, calculate_dat_score
from request import Request, run_request
from scipy.stats import norm
from datetime import datetime
from creative_tests.DAT import DivergentAssociationTest

test = DivergentAssociationTest(
    models=[
        "gemini-2.5-flash-preview-05-20",
        "gemini-2.5-flash-preview-04-17",
        ],
    configs=[{"temperature": 1.5, "thinking_config": {"thinking_budget": 512}}],
    repeats=2,
    delay=6,
)

test.run()

Number of API calls: 4
Estimated Time: 0.43333333333333335
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.5, thinking_config={'thinking_budget': 512}} (1)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.5, thinking_config={'thinking_budget': 512}} (2)...
Requesting gemini-2.5-flash-preview-04-17 with config={temperature=1.5, thinking_config={'thinking_budget': 512}} (1)...


KeyboardInterrupt: 

In [1]:
import os
import json
import numpy as np
from google import genai
from google.genai import types
from langchain_text_splitters import RecursiveCharacterTextSplitter
from embeddings import GloVe, calculate_dat_score
from request import Request, run_request
from scipy.stats import norm
from datetime import datetime
from creative_tests.DAT import DivergentAssociationTest

test = DivergentAssociationTest(
    models=[
        "gemini-2.5-flash-preview-05-20",
        "gemini-2.5-flash-preview-04-17",
        ],
    configs=[{"temperature": 1.0, "thinking_config": {"thinking_budget": 100}}, 
             {"temperature": 1.0, "thinking_config": {"thinking_budget": 512}},
             {"temperature": 1.0, "thinking_config": {"thinking_budget": 1000}},
             {"temperature": 1.0, "thinking_config": {"thinking_budget": 2000}}],
    repeats=10,
    delay=10,
)

test.run()

Number of API calls: 80
Estimated Time: 14.0
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (1)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (2)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (3)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (4)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (5)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (6)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 100}} (7)...
Requesting gemini-2.5-flash-preview-05-20 with config={temperature=1.0, thinking_config={'thinking_budget': 